<a href="https://colab.research.google.com/github/parinnay/quant_finance/blob/main/value_based_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [2]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 4.3 MB/s 


In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [4]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [5]:
symbol = "AAPL"
api_url = api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 96764688,
 'calculationPrice': 'close',
 'change': -0.3,
 'changePercent': -0.00177,
 'close': 175.56,
 'closeSource': 'lcoifiaf',
 'closeTime': 1650971000309,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 176.983,
 'delayedPriceTime': 1714458456698,
 'extendedChange': -0.32,
 'extendedChangePercent': -0.00186,
 'extendedPrice': 182,
 'extendedPriceTime': 1679240504697,
 'high': 180.07,
 'highSource': 'e eteded 15lnirmip ucya',
 'highTime': 1713590525939,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 178.42,
 'iexCloseTime': 1695699385563,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 181.673,
 'iexOpenTime': 1695459629590,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1660817393958,
 'latestPrice': 175.82,
 'latestSource': 'Close',
 'latestTime': 'April 1, 2022',
 'latestUpdate': 1712398439922,
 'latestVolu

In [6]:
pe_ratio = data["peRatio"]
pe_ratio

29.93

In [7]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

my_columns = ["Ticker", "Price", "Price-to-Earnings Ratio", "Number of Shares to Buy"]

In [8]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        try:
            final_df = final_df.append(
                pd.Series([symbol,
                        data[symbol]["quote"]["latestPrice"],
                        data[symbol]["quote"]["peRatio"],
                        "N/A"],
                        index = my_columns), 
                        ignore_index = True
            )
        except:
            pass

final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.37,34,N/A
1,AAL,18.97,-6,N/A
2,AAP,212.71,22.33,N/A
3,AAPL,175.98,29.1,N/A
4,ABBV,164.16,26.07,N/A
...,...,...,...,...
497,YUM,121.29,23.33,N/A
498,ZBH,128.94,68.8,N/A
499,ZBRA,426.32,26.68,N/A
500,ZION,66.60,6.55,N/A


In [9]:
final_df.sort_values("Price-to-Earnings Ratio", inplace = True)
final_df = final_df[final_df["Price-to-Earnings Ratio"] > 0]
final_df = final_df[:50]
final_df.reset_index(inplace = True)
final_df.drop("index", axis = 1, inplace = True)

In [10]:
final_df.sort_values(by = "Price-to-Earnings Ratio", ascending = False, inplace = True)
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
49,KSS,61.18,9.97,N/A
48,WU,19.94,9.94,N/A
47,CTL,11.00,9.89,N/A
46,MU,78.88,9.83,N/A
45,REGN,716.28,9.81,N/A
44,BBY,93.75,9.55,N/A
43,TSN,91.22,9.34,N/A
42,RE,319.50,9.28,N/A
41,LNC,67.33,9.17,N/A
40,CFG,45.11,8.99,N/A


In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [12]:
portfolio_input()

Enter the value of your portfolio:100000


In [13]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df['Price'][i])
final_df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
49,KSS,61.18,9.97,32
48,WU,19.94,9.94,100
47,CTL,11.00,9.89,181
46,MU,78.88,9.83,25
45,REGN,716.28,9.81,2
44,BBY,93.75,9.55,21
43,TSN,91.22,9.34,21
42,RE,319.50,9.28,6
41,LNC,67.33,9.17,29
40,CFG,45.11,8.99,44


In [15]:
#improved strategy

In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [17]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

In [19]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            
            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN
            
            try:
                ev_to_gross_profit = enterprise_value/gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN
                
            rv_dataframe = rv_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
            ],
            index = rv_columns),
                ignore_index = True
            )
        except:
            pass

In [20]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.27,N/A,35,N/A,7.83,N/A,6.47,N/A,25.003309,N/A,12.346191,N/A,N/A
1,AAL,18.64,N/A,-6,N/A,-1.61,N/A,0.4123,N/A,-11.779731,N/A,1.197344,N/A,N/A
2,AAP,212.06,N/A,22.71,N/A,4.06,N/A,1.18,N/A,12.321521,N/A,2.775317,N/A,N/A
3,AAPL,175.96,N/A,29.78,N/A,39.9,N/A,7.77,N/A,23.185203,N/A,18.527782,N/A,N/A
4,ABBV,167.68,N/A,26.42,N/A,19.29,N/A,5.2,N/A,12.871122,N/A,9.370086,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,YUM,119.36,N/A,23.41,N/A,-4.2,N/A,5.26,N/A,19.662102,N/A,9.093426,N/A,N/A
717,ZBH,131.72,N/A,69.7,N/A,2.16,N/A,3.52,N/A,19.036049,N/A,6.021090,N/A,N/A
718,ZBRA,429.45,N/A,27.34,N/A,7.42,N/A,4,N/A,19.217707,N/A,8.579975,N/A,N/A
719,ZION,66.40,N/A,6.37,N/A,1.44,N/A,2.85,N/A,4.822652,N/A,2.561124,N/A,N/A


In [21]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [22]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [23]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [24]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.27,N/A,35.00,0.73301,7.83,0.789182,6.4700,0.811373,25.003309,0.807212,12.346191,0.803051,N/A
1,AAL,18.64,N/A,-6.00,0.038141,-1.61,0.051318,0.4123,0.026352,-11.779731,0.054092,1.197344,0.034674,N/A
2,AAP,212.06,N/A,22.71,0.471567,4.06,0.575589,1.1800,0.149098,12.321521,0.402219,2.775317,0.138696,N/A
3,AAPL,175.96,N/A,29.78,0.643551,39.90,0.972261,7.7700,0.850208,23.185203,0.769764,18.527782,0.918169,N/A
4,ABBV,167.68,N/A,26.42,0.560333,19.29,0.932039,5.2000,0.723301,12.871122,0.427184,9.370086,0.669903,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,YUM,119.36,N/A,23.41,0.483356,-4.20,0.04577,5.2600,0.729542,19.662102,0.692094,9.093426,0.647712,N/A
717,ZBH,131.72,N/A,69.70,0.930652,2.16,0.278779,3.5200,0.52982,19.036049,0.672677,6.021090,0.393897,N/A
718,ZBRA,429.45,N/A,27.34,0.578363,7.42,0.764216,4.0000,0.588766,19.217707,0.676838,8.579975,0.567268,N/A
719,ZION,66.40,N/A,6.37,0.09154,1.44,0.149792,2.8500,0.436893,4.822652,0.102635,2.561124,0.127601,N/A


In [25]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.27,N/A,35.00,0.73301,7.83,0.789182,6.4700,0.811373,25.003309,0.807212,12.346191,0.803051,0.788766
1,AAL,18.64,N/A,-6.00,0.038141,-1.61,0.051318,0.4123,0.026352,-11.779731,0.054092,1.197344,0.034674,0.040915
2,AAP,212.06,N/A,22.71,0.471567,4.06,0.575589,1.1800,0.149098,12.321521,0.402219,2.775317,0.138696,0.347434
3,AAPL,175.96,N/A,29.78,0.643551,39.90,0.972261,7.7700,0.850208,23.185203,0.769764,18.527782,0.918169,0.830791
4,ABBV,167.68,N/A,26.42,0.560333,19.29,0.932039,5.2000,0.723301,12.871122,0.427184,9.370086,0.669903,0.662552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,YUM,119.36,N/A,23.41,0.483356,-4.20,0.04577,5.2600,0.729542,19.662102,0.692094,9.093426,0.647712,0.519695
717,ZBH,131.72,N/A,69.70,0.930652,2.16,0.278779,3.5200,0.52982,19.036049,0.672677,6.021090,0.393897,0.561165
718,ZBRA,429.45,N/A,27.34,0.578363,7.42,0.764216,4.0000,0.588766,19.217707,0.676838,8.579975,0.567268,0.63509
719,ZION,66.40,N/A,6.37,0.09154,1.44,0.149792,2.8500,0.436893,4.822652,0.102635,2.561124,0.127601,0.181692


In [26]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [27]:
portfolio_input()

Enter the value of your portfolio:100000


In [28]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,18.68,107,-6.00,0.038141,-1.6200,0.049931,0.4110,0.024965,-12.568768,0.049931,1.258193,0.040222,0.040638
1,AAL,18.64,107,-6.00,0.038141,-1.6100,0.051318,0.4123,0.026352,-11.779731,0.054092,1.197344,0.034674,0.040915
2,UNM,32.49,61,8.14,0.110957,0.5880,0.055479,0.5408,0.04577,3.952502,0.081831,0.529898,0.001387,0.059085
3,PRU,124.30,16,6.11,0.080444,0.7241,0.059639,0.6348,0.0638,4.143162,0.085992,0.638217,0.006935,0.059362
4,AIG,65.99,30,6.00,0.073509,0.7892,0.062413,1.0120,0.120666,3.897596,0.080444,1.016938,0.022191,0.071845
5,AIG,64.78,30,6.10,0.078363,0.7958,0.0638,1.0300,0.124133,3.738168,0.074896,0.961426,0.019417,0.072122
6,GM,43.81,45,6.58,0.0957,1.0500,0.083218,0.5029,0.044383,2.209253,0.061026,1.872425,0.084605,0.073786
7,PVH,78.43,25,5.93,0.072122,1.0385,0.079057,0.5983,0.054092,5.381895,0.127601,1.201669,0.036061,0.073786
8,GM,43.09,46,6.70,0.098474,1.0700,0.090846,0.4953,0.042996,2.203673,0.059639,1.939841,0.090153,0.076422
9,LNC,67.48,29,8.90,0.128988,0.5709,0.054092,0.6068,0.056865,6.423720,0.153953,0.570689,0.004161,0.079612


In [29]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


In [31]:
writer.save()